In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import scipy
import scipy.io
import argparse
import tensorflow as tf

# parser = argparse.ArgumentParser()
# parser.add_argument('--ld', type=float, default=500000) # lambda
# parser.add_argument("-f", "--file", required=False)
# opts = vars(parser.parse_args())

In [ ]:
class Args:
  ld = 500000.0

opts = Args

In [ ]:
def normalizeFeature(x):
	# x = d x N dims (d: feature dimension, N: the number of features)
	x = x + 1e-10 # for avoid RuntimeWarning: invalid value encountered in divide
	feature_norm = np.sum(x**2, axis=1)**0.5 # l2-norm
	feat = x / feature_norm[:, np.newaxis]
	return feat

In [ ]:
def SAE(x, s, ld):
	# SAE is Semantic Autoencoder
	# INPUTS:
	# 	x: d x N data matrix
	#	s: k x N semantic matrix
	#	ld: lambda for regularization parameter
	#
	# OUTPUT:
	#	w: kxd projection matrix

	A = np.dot(s, s.transpose())
	B = ld * np.dot(x, x.transpose())
	C = (1+ld) * np.dot(s, x.transpose())
	w = scipy.linalg.solve_sylvester(A,B,C)
	return w

In [ ]:
def distCosine(x, y):
	xx = np.sum(x**2, axis=1)**0.5
	x = x / xx[:, np.newaxis]
	yy = np.sum(y**2, axis=1)**0.5
	y = y / yy[:, np.newaxis]
	dist = 1 - np.dot(x, y.transpose())
	return dist

In [ ]:
def zsl_acc(semantic_predicted, semantic_gt, opts):
	# zsl_acc calculates zero-shot classification accruacy
	#
	# INPUTS:
	#	semantic_prediced: predicted semantic labels
	# 	semantic_gt: ground truth semantic labels
	# 	opts: other parameters
	#
	# OUTPUT:
	# 	zsl_accuracy: zero-shot classification accuracy (per-sample)

	dist = 1 - distCosine(semantic_predicted, semantic_gt)
	y_hit_k = np.zeros((dist.shape[0], opts.HITK))
	for idx in range(0, dist.shape[0]):
		sorted_id = sorted(range(len(dist[idx,:])), key=lambda k: dist[idx,:][k], reverse=True)
		y_hit_k[idx,:] = opts.test_classes_id[sorted_id[0:opts.HITK]]

	n = 0
	for idx in range(0, dist.shape[0]):
		if opts.test_labels[idx] in y_hit_k[idx,:]:
			n = n + 1
	zsl_accuracy = float(n) / dist.shape[0] * 100
	return zsl_accuracy, y_hit_k

In [ ]:
awa = scipy.io.loadmat('/content/drive/MyDrive/Text_Embedding_Academic_project/mat_files/all_files.mat')
train_data = awa['X_tr']
train_data = normalizeFeature(train_data.transpose()).transpose()
test_data = awa['X_te3']
test_data = normalizeFeature(test_data.transpose()).transpose()
train_class_attributes = awa['S_tr']
opts.test_labels = awa['test_labels3']
opts.test_classes_id = awa['testclasses_id']
test_class_attributes_labels_continuous = awa['S_te_gt']

In [ ]:
opts.test_labels.shape

(16353, 1)

In [ ]:
test_data.shape

(16353, 11)

In [ ]:
train_data.shape

(237658, 11)

In [ ]:
np.unique(opts.test_labels)

array([3], dtype=uint8)

In [ ]:
train_data

array([[2.79421185e-03, 3.30033003e-03, 1.22095572e-03, ...,
        3.59767677e-06, 5.99940237e-18, 4.92663025e-04],
       [2.97206576e-03, 1.65016502e-13, 2.44191143e-03, ...,
        2.69825757e-05, 5.99940237e-18, 3.44864117e-03],
       [1.47964117e-05, 3.30033003e-03, 1.22095572e-03, ...,
        3.17794781e-05, 3.93170835e-03, 4.92663025e-04],
       ...,
       [8.64634884e-04, 3.30033003e-03, 1.22095572e-03, ...,
        9.95357239e-05, 5.99940237e-18, 4.92663025e-04],
       [3.47504041e-03, 1.65016502e-13, 2.44191143e-03, ...,
        3.89748316e-05, 5.99940237e-18, 3.44864117e-03],
       [2.34464252e-03, 3.30033003e-03, 1.22095572e-03, ...,
        1.97872222e-05, 5.99940237e-18, 4.92663025e-04]])

In [ ]:
train_class_attributes.shape

(237658, 50)

In [ ]:
train_class_attributes

array([[-0.02348983,  0.31598322, -0.13860836, ..., -0.76849509,
         1.23180037,  0.39720472],
       [-0.02348983,  0.31598322, -0.13860836, ..., -0.76849509,
         1.23180037,  0.39720472],
       [-0.02348983,  0.31598322, -0.13860836, ..., -0.76849509,
         1.23180037,  0.39720472],
       ...,
       [-0.02916017,  0.41163409, -0.17977458, ..., -1.03785024,
         1.63478557,  0.54848268],
       [-0.02916017,  0.41163409, -0.17977458, ..., -1.03785024,
         1.63478557,  0.54848268],
       [-0.02916017,  0.41163409, -0.17977458, ..., -1.03785024,
         1.63478557,  0.54848268]])

In [ ]:
denseModel = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation = 'relu', input_shape=(11,)),
    tf.keras.layers.Dense(512, activation = 'relu'),
    #tf.keras.layers.Dense(256, activation = 'relu'),
    #tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(50)
		])

denseModel.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

denseModel.fit(train_data, train_class_attributes, epochs = 10, batch_size=32)

opts.HITK = 1

#[F --> S], projecting data from feature space to semantic space: 84.68% for AwA dataset
semantic_predicted = denseModel.predict(test_data)
[zsl_accuracy, y_hit_k] = zsl_acc(semantic_predicted, test_class_attributes_labels_continuous, opts)
print('[1] zsl accuracy for AwA dataset [F >>> S]: {:.2f}%'.format(zsl_accuracy))



In [ ]:
semantic_predicted

array([[-0.05836044,  0.36350268, -0.21859963, ..., -1.0093279 ,
         1.715552  ,  0.44132665],
       [-0.05836028,  0.3634993 , -0.2186003 , ..., -1.009321  ,
         1.7155405 ,  0.44132102],
       [-0.05831223,  0.36346996, -0.21863535, ..., -1.0094366 ,
         1.7159401 ,  0.4413401 ],
       ...,
       [-0.05831227,  0.3634764 , -0.21863388, ..., -1.0094501 ,
         1.7159623 ,  0.4413512 ],
       [-0.05831226,  0.3634764 , -0.21863388, ..., -1.0094502 ,
         1.7159623 ,  0.44135118],
       [-0.05831227,  0.3634764 , -0.21863389, ..., -1.0094501 ,
         1.7159623 ,  0.44135118]], dtype=float32)

In [ ]:
train_data_reshaped = np.expand_dims(train_data, axis=2)
print(train_data_reshaped.shape)

Str_data_reshaped = np.expand_dims(train_class_attributes, axis=2)
print(Str_data_reshaped.shape)

(237658, 11, 1)
(237658, 50, 1)


In [ ]:
nnModel = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(11, 1)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    #tf.keras.layers.Conv1D(filters = 64, kernel_size=3, activation='relu'),
    #tf.keras.layers.MaxPooling1D(pool_size=2),
    #tf.keras.layers.Conv1D(filters = 64, kernel_size=3, activation='relu'),
    #tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(50)
])

In [ ]:
nnModel.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
nnModel.fit(train_data_reshaped, Str_data_reshaped, epochs=10, batch_size=32)

Epoch 1/10
7427/7427 [==============================] - 32s 3ms/step - loss: 0.0111
Epoch 2/10
7427/7427 [==============================] - 40s 5ms/step - loss: 0.0079
Epoch 3/10
7427/7427 [==============================] - 24s 3ms/step - loss: 0.0078
Epoch 4/10
7427/7427 [==============================] - 25s 3ms/step - loss: 0.0078
Epoch 5/10
7427/7427 [==============================] - 25s 3ms/step - loss: 0.0078
Epoch 6/10
7427/7427 [==============================] - 26s 4ms/step - loss: 0.0078
Epoch 7/10
7427/7427 [==============================] - 25s 3ms/step - loss: 0.0077
Epoch 8/10
7427/7427 [==============================] - 24s 3ms/step - loss: 0.0077
Epoch 9/10
7427/7427 [==============================] - 24s 3ms/step - loss: 0.0077
Epoch 10/10
7427/7427 [==============================] - 25s 3ms/step - loss: 0.0077


In [ ]:
opts.HITK = 1
#[F --> S], projecting data from feature space to semantic space: 84.68% for AwA dataset
semantic_predicted = nnModel.predict(test_data)
[zsl_accuracy, y_hit_k] = zsl_acc(semantic_predicted, test_class_attributes_labels_continuous, opts)
print('[1] zsl accuracy for AwA dataset [F >>> S]: {:.2f}%'.format(zsl_accuracy))

29/29 [==============================] - 0s 2ms/step
[1] zsl accuracy for AwA dataset [F >>> S]: 98.55%


In [ ]:
train_ratio = 0.8
train_samples = int(24295 * train_ratio)

train_input = train_data[:train_samples]
train_output = train_class_attributes[:train_samples]
val_input = train_data[train_samples:]
val_output = train_class_attributes[train_samples:]

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(11,)),
    #tf.keras.layers.Dense(256, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    #tf.keras.layers.Dense(256, activation = 'relu'),
    #tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(50)
		])

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_input, train_output, epochs=10, batch_size=32, validation_data=(val_input, val_output))

Epoch 1/10
608/608 [==============================] - 24s 37ms/step - loss: 0.0403 - accuracy: 0.9895 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 2/10
608/608 [==============================] - 13s 22ms/step - loss: 5.4103e-09 - accuracy: 1.0000 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 3/10
608/608 [==============================] - 23s 37ms/step - loss: 1.1945e-08 - accuracy: 1.0000 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 4/10
608/608 [==============================] - 13s 22ms/step - loss: 1.0263e-08 - accuracy: 1.0000 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 5/10
608/608 [==============================] - 13s 22ms/step - loss: 1.3879e-08 - accuracy: 1.0000 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 6/10
608/608 [==============================] - 13s 22ms/step - loss: 1.9627e-08 - accuracy: 1.0000 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 7/10
608/608 [==============================] - 13s 22ms/step - loss: 1.5407e-08 - accuracy: 1.0000 - val_loss

In [ ]:
opts.HITK = 1
#[F --> S], projecting data from feature space to semantic space: 84.68% for AwA dataset
semantic_predicted = model.predict(test_data)
[zsl_accuracy, y_hit_k] = zsl_acc(semantic_predicted, test_class_attributes_labels_continuous, opts)
print('[1] zsl accuracy for AwA dataset [F >>> S]: {:.2f}%'.format(zsl_accuracy))

29/29 [==============================] - 0s 1ms/step
[1] zsl accuracy for AwA dataset [F >>> S]: 100.00%


In [ ]:
# Encoder
encoder_inputs = tf.keras.Input((11,))
encoder = tf.keras.layers.Dense(256, activation='relu')(encoder_inputs)
#encoder = tf.keras.layers.Dense(128, activation='relu')(encoder)
encoded_outputs = tf.keras.layers.Dense(50)(encoder)

# Decoder
#decoder = tf.keras.layers.Dense(128, activation='relu')(encoded_outputs)
decoder = tf.keras.layers.Dense(256, activation='relu')(encoded_outputs)
decoder_outputs = tf.keras.layers.Dense(50)(decoder)

In [ ]:
# Autoencoder model
autoencoder = tf.keras.Model(encoder_inputs, decoder_outputs)

In [ ]:
# Compile the autoencoder model with suitable optimizer and loss function
autoencoder.compile(optimizer='adam', loss='mean_squared_error',  metrics=['accuracy'])

In [ ]:
# Train the autoencoder on the input-output data
autoencoder.fit(train_input, train_output, epochs=10, batch_size=32, validation_data=(val_input, val_output))

Epoch 1/10
608/608 [==============================] - 16s 23ms/step - loss: 0.0181 - accuracy: 0.9983 - val_loss: 0.0127 - val_accuracy: 1.0000
Epoch 2/10
608/608 [==============================] - 22s 37ms/step - loss: 1.3440e-07 - accuracy: 1.0000 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 3/10
608/608 [==============================] - 23s 37ms/step - loss: 4.9294e-08 - accuracy: 1.0000 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 4/10
608/608 [==============================] - 23s 37ms/step - loss: 5.6635e-08 - accuracy: 1.0000 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 5/10
608/608 [==============================] - 13s 22ms/step - loss: 6.4883e-08 - accuracy: 1.0000 - val_loss: 0.0128 - val_accuracy: 1.0000
Epoch 6/10
608/608 [==============================] - 22s 37ms/step - loss: 7.2282e-08 - accuracy: 1.0000 - val_loss: 0.0127 - val_accuracy: 1.0000
Epoch 7/10
608/608 [==============================] - 22s 37ms/step - loss: 9.3909e-08 - accuracy: 1.0000 - val_loss

In [ ]:
opts.HITK = 1
#[F --> S], projecting data from feature space to semantic space: 84.68% for AwA dataset
semantic_predicted = autoencoder.predict(test_data)
[zsl_accuracy, y_hit_k] = zsl_acc(semantic_predicted, test_class_attributes_labels_continuous, opts)
print('[1] zsl accuracy for AwA dataset [F >>> S]: {:.2f}%'.format(zsl_accuracy))

29/29 [==============================] - 0s 1ms/step
[1] zsl accuracy for AwA dataset [F >>> S]: 99.22%
